# Agent V4: The Coordinator

A monolithic agent with sub-agent delegation. Instead of giving the agent 40+ specialized
tools, it declares sub-agents via `AgentSpec(sub_agents=[...])`. 

The `OrchestratorAgent` auto-creates a generic `delegate(agent_name, prompt)` tool from the sub-agent specs.

Extends the Skilled agent: 
- file tools
- sandbox
- todo
- skill
- delegate
- conversion

**Prerequisite:** Run `scripts/db_ingest_bookstore_sqlite.sh` to create the bookstore database before running this notebook.

In [ ]:
import shutil

from agentic_patterns.core.agents import AgentSpec, OrchestratorAgent
from agentic_patterns.core.config.config import PROMPTS_DIR, WORKSPACE_DIR
from agentic_patterns.core.prompt import load_prompt
from agentic_patterns.core.user_session import get_user_id, get_session_id
from agentic_patterns.tools.file import get_all_tools as get_file_tools
from agentic_patterns.tools.sandbox import get_all_tools as get_sandbox_tools
from agentic_patterns.tools.todo import get_all_tools as get_todo_tools
from agentic_patterns.tools.format_conversion import get_all_tools as get_format_conversion_tools
from agentic_patterns.agents.data_analysis import get_spec as get_data_analysis_spec
from agentic_patterns.agents.sql import get_spec as get_sql_spec
from agentic_patterns.agents.vocabulary import get_spec as get_vocabulary_spec

## Agent Definition

The coordinator extends the Skilled agent with sub-agent delegation via `AgentSpec(sub_agents=[...])`.
The `OrchestratorAgent` auto-creates a `delegate(agent_name, prompt)` tool from the specs.
Skills are still auto-discovered by the `OrchestratorAgent`.

In [ ]:
system_prompt = load_prompt(PROMPTS_DIR / "the_complete_agent" / "agent_coordinator.md")

spec = AgentSpec(
    name="coordinator",
    system_prompt=system_prompt,
    tools=(
        get_file_tools()
        + get_sandbox_tools()
        + get_todo_tools()
        + get_format_conversion_tools()
    ),
    sub_agents=[
        get_data_analysis_spec(),
        get_sql_spec(),
        get_vocabulary_spec(),
    ],
)
print(spec)

In [ ]:
workspace_dir = WORKSPACE_DIR / get_user_id() / get_session_id()
if workspace_dir.exists():
    shutil.rmtree(workspace_dir)

# Clear private data flag from previous runs
from agentic_patterns.core.compliance import PrivateData
pd = PrivateData(get_user_id(), get_session_id())
pd.has_private_data = False
pd.save()

agent = OrchestratorAgent(spec, verbose=True)

## System Prompt

The `OrchestratorAgent` builds the final system prompt by combining the base prompt,
sub-agent descriptions (for the `delegate` tool), and skill listings. The prompt is
assembled during `__aenter__` and available via the `system_prompt` property.

In [ ]:
async with agent:
    print(agent.system_prompt)

## Turn 1: Query the Database

Ask about data in the bookstore database. The coordinator should delegate to `ask_sql_analyst`
rather than trying to query the database directly (it has no SQL tools of its own).

In [ ]:
async with agent:
    prompt_1 = """Query the bookstore database: how many books are there,
what genres are represented, and what is the average price per genre?
Save the per-genre results to a CSV file."""

    result_1 = await agent.run(prompt_1)

    print("\n--- Agent Output ---")
    print(result_1.output)

## Turn 2: Write a Report and Convert

Ask the agent to write a markdown report from the results and convert it to HTML.
This uses the coordinator's own file tools (to write the markdown) and
`convert_document` (to produce HTML). No delegation needed for this turn.

In [ ]:
async with agent:
    prompt_2 = """Write a markdown report summarizing the bookstore data from the previous query.
Include a bar chart showing the average price per genre (save it as /workspace/avg_price_by_genre.png
and embed it in the report). Save the report to /workspace/bookstore_report.md, then convert it to PDF."""

    result_2 = await agent.run(prompt_2)

    print("\n--- Agent Output ---")
    print(result_2.output)

## Verify Workspace

In [ ]:
from agentic_patterns.core.workspace import list_workspace_files, read_from_workspace

for path in sorted(list_workspace_files("*")):
    try:
        content = read_from_workspace(path)
    except UnicodeDecodeError:
        content = "<binary file>"
    print(f"--- {path} ---")
    print(content)
    print()